In [3]:
import os
print os.getcwd()
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")
#sys.path.append("../pylof-master/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
#features_list = ['poi','salary',''] # You will need to use more features

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
# 异常值， 去掉total
def remove_outerliers(data_dict,names):
    for name in names:
        data_dict.pop(name, 0)
    return data_dict

def dateset_summary(data_dic):
    n_poi,n_non_poi = 0,0
    for key in data_dict.keys():
        if data_dict[key]['poi']==1:
            n_poi+=1
        else:
            n_non_poi+=1        
    print "Total number of data points: ", len(data_dict)
    print "Number of POI: %d, no. of non-POI: %d"% (n_poi, n_non_poi)
    print "Number of features used: ",len(data_dict['METTS MARK']) # randomly pick one name, get the number of features.

outliers_names = ['TOTAL']#,'LAVORATO JOHN J'

data_dict = remove_outerliers(data_dict,outliers_names)
dateset_summary(dateset_summary)

import pandas as pd
import numpy as np

data_df = pd.DataFrame.from_dict(data_dict,orient='index')

financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus'
                 , 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses'
                 , 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock'
                 , 'director_fees']
email_fatures = ['to_messages','from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi'
                 , 'shared_receipt_with_poi']  # remove 'email_address'
def NaNs_to_0s(col):
    return [0 if ele=='NaN' else ele for ele in col]

def NaNs_to_None(col):
    return [None if ele=='NaN' else ele for ele in col]

def count_NAN(col):
    return sum(1 for ele in col if ele == 'NaN')

data_df[financial_features] = data_df[financial_features].apply(NaNs_to_0s, axis=0)
data_df['poi'] = data_df['poi'].astype(int)
print data_df.apply(count_NAN, axis=0)   

/Users/weidian1/Documents/GitHub/Udacity-DAND/P5-Intro to ML/Project5/final_project
Total number of data points:  145
Number of POI: 18, no. of non-POI: 127
Number of features used:  21
salary                        0
to_messages                  59
deferral_payments             0
total_payments                0
exercised_stock_options       0
bonus                         0
restricted_stock              0
shared_receipt_with_poi      59
restricted_stock_deferred     0
total_stock_value             0
expenses                      0
loan_advances                 0
from_messages                59
other                         0
from_this_person_to_poi      59
poi                           0
director_fees                 0
deferred_income               0
long_term_incentive           0
email_address                34
from_poi_to_this_person      59
dtype: int64


In [4]:
"""
make new feature 
1. 'email_features_miss': if email_features is missing. next step, missing email_features will be
transformed to 0, which should by different from missing financial features.   
2. poi_rate_to_messages = from_this_person_to_poi/to_messages  
3. poi_rate_from_messages = from_poi_to_this_person/from_messages

"""
data_df['email_features_miss'] = data_df.apply(lambda x: 1 if x["to_messages"]=='NaN' else 0, axis = 1)

data_df['poi_rate_to_messages'] = data_df.apply(lambda x: 0 if x['from_this_person_to_poi']=='NaN' \
                                                else x['from_this_person_to_poi']*1.0/x['to_messages'], axis=1)

data_df['poi_rate_from_messages'] = data_df.apply(lambda x: 0 if x['from_poi_to_this_person']=='NaN'\
                                                  else x['from_poi_to_this_person']*1.0/x['from_messages'], axis =1)



    

In [5]:
features_full_list = list(data_df.columns.values)
features_full_list.remove('email_address')
features_full_list.remove('poi')
features_full_list = ['poi'] +features_full_list
data_full_dic = data_df.to_dict(orient='index')
print ("%d full_columns_name: %s" % (len(features_full_list), features_full_list))

#def get_train_test_dataset(my_dataset,features_list):
#    #Extract features and labels from dataset for local testing
#    data = featureFormat(my_dataset, features_list, sort_keys = True)
#    labels, features = targetFeatureSplit(data)
#    # TODO: Shuffle the data
#    from sklearn.utils import shuffle
#    features, labels = shuffle(features, labels, random_state=1)
#    
#    # split train test dataset
#    from sklearn.model_selection import train_test_split
#    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)
#    X_train = np.array(X_train)
#    y_train = np.array(y_train)
#    X_test = np.array(X_test)
#    y_test = np.array(y_test)
#    
#    return X_train, X_test, y_train, y_test
#    
#X_train, X_test, y_train, y_test = get_train_test_dataset(data_full_dic,features_full_list)  
    
def get_features_labels(my_dataset,features_list):
    #Extract features and labels from dataset for local testing
    data = featureFormat(my_dataset, features_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    # TODO: Shuffle the data
    from sklearn.utils import shuffle
    features, labels = shuffle(features, labels, random_state=1)
    features = np.array(features)
    labels = np.array(labels)
    return features, labels

def get_train_test_dataset(features, labels):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    return X_train, X_test, y_train, y_test
  
features, labels = get_features_labels(data_full_dic,features_full_list)
X_train, X_test, y_train, y_test = get_train_test_dataset(features, labels)

23 full_columns_name: ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person', 'email_features_miss', 'poi_rate_to_messages', 'poi_rate_from_messages']


In [12]:
import pandas as pd
from numpy import mean,std
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import (ExtraTreesClassifier, RandomForestClassifier, 
                              AdaBoostClassifier, GradientBoostingClassifier)

from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.preprocessing import MinMaxScaler




def scorer_r_p(estimator, X_test, y_test):
    y_pred = estimator.predict(X_test)
    r_score = recall_score(y_test, y_pred)
    p_score = precision_score(y_test, y_pred)
#    f1_score = f1_score(y_test, y_pred)
    if r_score<0.3 or p_score<0.3:
        return 0    
    return r_score + p_score
 

class EstimatorSelectionHelper:
    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
        
        min_max_scaler = MinMaxScaler(feature_range=(0, 1))
        model_tree = SelectFromModel(ExtraTreesClassifier(random_state=32), prefit=False)
        preparation = [('scaler',min_max_scaler)
                      ,('selector',model_tree)]
        self.preparation = preparation
        self.params_selector ={'selector__threshold':['mean','0.5*mean','1.4*mean'] }
        self.scores_df = None

    def fit(self, X, y, cv=3, n_jobs=1, verbose=1, scoring=None, refit=True):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            #params = self.params[key]
            print params_selector.items()
            if isinstance(self.params[key],list):
                params = [dict(param.items() + params_selector.items()) for param in self.params[key]]           
            elif isinstance(self.params[key],dict):
                params = dict(self.params[key].items() + self.params_selector.items())
            preparation = self.preparation
            pipe= Pipeline(preparation +[(key,model)])        
            gs = GridSearchCV(pipe, params, cv=cv, n_jobs=n_jobs,verbose=verbose, scoring=scoring, refit=refit)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': mean(scores),
                 'std_score': std(scores),
            }
            return pd.Series(dict(params.items() + d.items()))

        rows = [row(k, gsc.cv_validation_scores, gsc.parameters) 
                     for k in self.keys
                     for gsc in self.grid_searches[k].grid_scores_]
        df = pd.concat(rows, axis=1).T.sort_values(by=[sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        self.scores_df = df[columns]
        return df[columns]
    
    def best_estimator_model(self):
        if  isinstance(self.scores_df,pd.DataFrame):
            model_name = self.scores_df.iloc[0,0]
            best_estimator_ = self.grid_searches[model_name].best_estimator_
            best_model = self.grid_searches[model_name].best_estimator_.steps[-1][-1]
            return best_estimator_, best_model       
        else:
            raise ValueError('scores_df is None; run fit() and score_summary() first')

In [13]:
models1 = { 
#    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
#    'AdaBoostClassifier': AdaBoostClassifier(),
#    'GradientBoostingClassifier': GradientBoostingClassifier()
    #'SVC': SVC()
}

params1 = { 
#    'ExtraTreesClassifier': { 'ExtraTreesClassifier__n_estimators': [16, 32, 50, 100] },
    'RandomForestClassifier': { 'RandomForestClassifier__n_estimators': [16, 32, 50, 100] }
#    'AdaBoostClassifier':  { 'AdaBoostClassifier__n_estimators': [16, 32, 50, 100] },
#    'GradientBoostingClassifier': { 'GradientBoostingClassifier__n_estimators': [16, 32, 50, 100]
#                                   , 'GradientBoostingClassifier__learning_rate': [0.5, 0.8, 1.0] }
    #'SVC': [
    #    {'SVC__kernel': ['linear'], 'SVC__C': [1, 10]},
    #    {'SVC__kernel': ['rbf'], 'SVC__C': [1, 10], 'SVC__gamma': [0.001, 0.0001]},
    #]
}


from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(labels, 1000, random_state=42)

helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(features, labels,cv=sss, scoring='recall', n_jobs=-1)

Running GridSearchCV for RandomForestClassifier.
[('selector__threshold', ['mean', '0.5*mean', '1.4*mean'])]
Fitting 1000 folds for each of 12 candidates, totalling 12000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.3s


KeyboardInterrupt: 